# Hi, in this notebook I will use crew AI and open ai to tailor Job Application resume by using multi agent system
## The notebook will also create a file with interview questions in a different file, named "interview_materials.md"

### you will need to put an SERPER_API_KEY and SERPER_API_KEY in the .env for this notebook
https://platform.openai.com/

https://serper.dev/

# this is the information that we will give to the project

In [3]:
# this is the information that we will give to the agents
job_posting_url = "  <type here>  "
github_url = "  <type here>  "
personal_writeup =  """  <type here> """

In [5]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## if this is your first time using this, or other notebooks with Crew AI you would need to download the needed modules

In [13]:
# if this is your first time using this, or other notebooks with Crew AI you would need to download the needed modules
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29


SyntaxError: invalid syntax (3139094060.py, line 2)

In [21]:
# import sys
# print(sys.executable)
# !conda install ipykernel -y
# !python -m ipykernel install --user --name=resumeTest --display-name "Python (resumeTest)"


In [28]:
# !pip show crewai crewai_tools langchain_community


# importing crew ai modules that we will need

In [24]:
# importing crew ai modules that we will need
from crewai import Agent, Task, Crew

ModuleNotFoundError: No module named 'crewai'

In [ ]:
from crewai import Agent, Task, Crew
print("Successfully imported crewai!")


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
# loading the api keys needed for this task
openai_api_key = os.getenv("OPENAI_API_KEY")
# the gpt-4o-mini open ai model is a lightweight and cheap model that i will be using in this notebook
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"
os.environ["SERPER_API_KEY"] =os.getenv("SERPER_API_KEY")

In [ ]:
# print(f"OpenAI API Key: {openai_api_key}")
# print(f"Serper API Key: {os.getenv('SERPER_API_KEY')}")


In [ ]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)
# this tool uses the Serper api to search google
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./resume.md')
# this tool enables Retrieval-Augmented Generation (RAG) by extracting relevant 
# information from the resume and feeding it to the model as needed
semantic_search_resume = MDXSearchTool(mdx='./resume.md')

# Creating the Agents that will transform the resume and make the interview questions

### Agent 1: Researcher

In [ ]:
# Agent 1: Researcher - this agent will research the job postings, 
# analyze the required qualifications, 
# and extract key skills needed for applicants.
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=False, # Set to "True" to enable detailed logs of the agent's operations if you are intrested
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

### Agent 2: Profiler

In [ ]:
# Agent 2: Profiler - This agent helps you, the job applicant, 
# stand out by analyzing your resume and professional info. 
# It scrapes job listings and performs semantic searches to gather insights, 
# helping personalize and enhance your resume for better job market visibility.
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=False,  # Set to "True" to enable detailed logs of the agent's operations if you are intrested
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

### Agent 3

In [ ]:
# Agent 3: Resume Strategist - This agent helps you optimize your resume to stand out in the job market.
# It analyzes job listings and your resume to highlight your most relevant skills and experiences, ensuring 
# your resume aligns perfectly with job requirements.
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=False, # Set to "True" to enable detailed logs of the agent's operations if you are intrested
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

### Agent 4

In [ ]:
# Agent 4: Interview Preparer - 
# This agent helps you prepare for interviews 
# by creating relevant questions and talking points 
# based on your resume and the job requirements.

interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=False, # Set to "True" to enable detailed logs of the agent's operations if you are intrested
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

# now we will create the tasks for the agents

### Task for Researcher Agent

In [ ]:
# Task for Researcher Agent: Extract Job Requirements - 
# This task involves analyzing the provided job posting URL to 
# extract and categorize key job requirements such as skills, 
# qualifications, and experiences.

research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

### Task for Profiler Agent

In [ ]:
# Task for Profiler Agent: Compile Comprehensive Profile - 
# This task involves creating a detailed personal and 
# professional profile using the provided GitHub URL 
# and personal write-up, extracting key information from these sources.
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

### Task for Resume Strategist Agent

In [ ]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements - 
# This task involves tailoring the resume based on the profile and job requirements, 
# updating all sections to highlight the most relevant qualifications and experiences, 
# without making up information.
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
        " Return just the resume, without ``` in the beggining and ``` This resume effectively highlights... text at the end of the resume please."
        "use only '# ', '## ', '### ', '- ', surrounding words in ** like: **<text>** and like *<text>* styles in the resume for the markdown styling please."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

### Task for Interview Preparer Agent

In [ ]:
# Task for Interview Preparer Agent: Develop Interview Materials - 
# This task creates interview questions and talking points based on 
# the tailored resume and job requirements, 
# helping the candidate prepare to highlight key points during the interview.
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


### Job Application Crew

In [ ]:
# Job Application Crew: This crew coordinates 
# the efforts of multiple agents to assist the job applicant in all stages, 
# from researching job requirements to preparing for the interview. 
# Each agent handles a specific task to optimize the application process.
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=False # Set to "True" to enable detailed logs of the crew's operations if you are intrested
)

In [ ]:

job_application_inputs = {
    'job_posting_url': job_posting_url,
    'github_url': github_url,
    'personal_writeup': personal_writeup
}

# this is where the magic starts

## this exexcutes the project

### this execution will take a few minutes to run

In [ ]:

result = job_application_crew.kickoff(inputs=job_application_inputs)

In [ ]:
# pip install python-docx


# this will transform the previosly created *tailored_resume.md* to word format *tailored_resume.docx*

In [ ]:
md_file = 'tailored_resume.md'
output_file = 'tailored_resume.docx'

In [ ]:
from docx import Document

def markdown_to_word(md_file, output_file):
    # Create a new Document object
    doc = Document()

    with open(md_file, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for line in lines:
        line = line.strip()

        # Heading 1 (H1: # or # **Bold Text**)
        if line.startswith('# '):
            heading = line[2:].strip()
            add_formatted_text(doc, heading, heading_level=1)

        # Heading 2 (H2: ## or ## **Bold Text**)
        elif line.startswith('## '):
            heading = line[3:].strip()
            add_formatted_text(doc, heading, heading_level=2)

        # Heading 3 (H3: ### or ### **Bold Text**)
        elif line.startswith('### '):
            heading = line[4:].strip()
            add_formatted_text(doc, heading, heading_level=3)

        # Bullet points (-), including "- **Bold Text:**"
        elif line.startswith('- '):
            paragraph = doc.add_paragraph(style='List Bullet')
            process_bold_and_italic(line[2:], paragraph)

        # Regular paragraph, handling **bold** and *italic*
        elif line:
            paragraph = doc.add_paragraph()
            process_bold_and_italic(line, paragraph)

    # Save the document
    doc.save(output_file)

def add_formatted_text(doc, text, heading_level):
    """Handles adding bold text for headings like # **Bold Text**."""
    paragraph = doc.add_heading(level=heading_level)
    process_bold_and_italic(text, paragraph)

def process_bold_and_italic(text, paragraph):
    """
    Processes bold (**text**) and italic (*text*) Markdown syntax
    and applies the appropriate Word formatting.
    """
    i = 0
    while i < len(text):
        if text[i:i+2] == '**':  # Bold start
            i += 2
            bold_start = i
            while i < len(text) - 1 and text[i:i+2] != '**':
                i += 1
            paragraph.add_run(text[bold_start:i]).bold = True
            i += 2  # Skip the closing **
        elif text[i] == '*':  # Italic start
            i += 1
            italic_start = i
            while i < len(text) and text[i] != '*':
                i += 1
            paragraph.add_run(text[italic_start:i]).italic = True
            i += 1  # Skip the closing *
        else:
            start = i
            while i < len(text) and text[i:i+2] not in ('**', '*'):
                i += 1
            paragraph.add_run(text[start:i])  # Regular text

# Example usage

markdown_to_word(md_file, output_file)
